In [1]:
import wmfdata
spark = wmfdata.spark.get_session('yarn-large')
sc=spark.sparkContext

import pyspark
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import timedelta, date
import matplotlib.pyplot as plt
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
import pandas as pd


DATA_FOLDER = "how_we_read_wikipedia_march/en"

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [2]:
aggregated_sessions = spark.read.parquet("{}/aggregated_sessions_final.parquet".format(DATA_FOLDER))\
                                .repartition(12000)
aggregated_sessions

DataFrame[access_method: string, country_code: string, session: array<struct<actual_destination:string,http_status:string,local_time:timestamp,page_id:bigint,page_title:string,prev_load:bigint,referer:string>>, timezone: string, user_identifier: string, previews: array<struct<http_status:string,local_time:timestamp,page_title:string,preview_title:string>>]

In [3]:
aggregated_sessions.count()

1479882553

In [4]:
def split_session_1h(row):
    sessions = []
    tmp = [row.session[0]]
    loaded_articles = set([row.session[0].page_id])
    for pl in row.session[1:]:
        if pl.prev_load>3600 and '.wikipedia.org/wiki/' not in pl.referer:
            sessions.append(tmp)
            tmp = []
            loaded_articles = set()
        if pl.page_id not in loaded_articles:
            tmp.append(pl)
            loaded_articles.add(pl.page_id)
    if len(tmp)>0:
        sessions.append(tmp)
    return [Row(user_identifier=row.user_identifier, access_method=row.access_method, 
               country_code=row.country_code, session1h=s) for s in sessions]

sessions1h = spark.createDataFrame(aggregated_sessions.rdd.flatMap(split_session_1h)).repartition(12000)
sessions1h = sessions1h.withColumn("session_id", monotonically_increasing_id())
sessions1h

DataFrame[access_method: string, country_code: string, session1h: array<struct<actual_destination:string,http_status:string,local_time:timestamp,page_id:bigint,page_title:string,prev_load:bigint,referer:string>>, user_identifier: string, session_id: bigint]

In [5]:
sessions1h.write.mode("overwrite").parquet("{}/reading_sessions1h.parquet".format(DATA_FOLDER))

In [6]:
sessions1h.count()

2576027156